# Backtesting.py 入門


In [21]:
from abc import ABC
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import algotrade as algo

## データ
Backtesting ingests data as a pandas.DataFrame with columns 'Open', 'High', 'Low', 'Close', and (optionally) 'Volume'. 
Such data is easily obtainable (see e.g. pandas-datareader, Quandl, findatapy, ...). 
Your data frames can have other columns, but these are necessary. 
DataFrame should ideally be indexed with a datetime index (convert it with pd.to_datetime()), 
otherwise a simple range index will do.


In [22]:
# データの読み込み
df = algo.read_fx("../data/DAT_ASCII_EURJPY_M1_201501_201906.csv", fill_time=True)
df.tail(2)

# データを1時間足に直す
df = df.resample('1d').ohlc2()
df

,Open,High,Low,Close,Volume
2015-01-01,144.776,145.217,144.770,145.162,0.0
2015-01-02,145.160,145.323,144.132,144.558,0.0
2015-01-03,144.546,144.566,144.546,144.558,0.0
2015-01-04,144.546,144.566,143.142,143.942,0.0
2015-01-05,143.941,144.161,142.265,142.479,0.0
...,...,...,...,...,...
2019-06-17,121.834,122.125,121.599,121.645,0.0
2019-06-18,121.645,121.765,121.064,121.399,0.0
2019-06-19,121.399,121.653,121.109,121.284,0.0
2019-06-20,121.285,121.921,120.976,121.039,0.0


## トレード戦略

In [24]:
# 戦略の定義
class SmaCross(Strategy, ABC):
    
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def __init__(self, broker, data):
        super().__init__(broker, data)
        # Precompute two moving averages
        self.sma_short = self.I(algo.sma, self.data.Close, self.n1) # 短期の移動平均線
        self.sma_long = self.I(algo.sma, self.data.Close, self.n2) # 長期の移動平均線
    
    def next(self):
        # sma_shortがsma_longを上回ったら、買い注文
        if crossover(self.sma_short, self.sma_long):
            self.buy()

       # sma_shortがsma_longを下回ったら、売り注文
        elif crossover(self.sma_long, self.sma_short):
            self.sell()


## バックテストを実行


In [25]:
bt = Backtest(df, SmaCross, cash=7000, commission=0)
bt.run()

Start                     2015-01-01 00:00:00
End                       2019-06-21 00:00:00
Duration                   1632 days 00:00:00
Exposure [%]                          97.6103
Equity Final [$]                      6156.58
Equity Peak [$]                       7575.26
Return [%]                           -12.0488
Buy & Hold Return [%]                 15.9635
Max. Drawdown [%]                    -18.7277
Avg. Drawdown [%]                    -2.80912
Max. Drawdown Duration      781 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                   91
Win Rate [%]                          35.1648
Best Trade [%]                        6.17149
Worst Trade [%]                      -3.86098
Avg. Trade [%]                      -0.121145
Max. Trade Duration          58 days 00:00:00
Avg. Trade Duration          18 days 00:00:00
Expectancy [%]                        1.13277
SQN                                  -0.74814
Sharpe Ratio                      

In [26]:
# バックテストの結果を可視化
bt.plot(plot_width= 600)

## 最適化

In [35]:
%%time

stats, heatmap = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 100, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2,
                    return_heatmap=True)

CPU times: user 184 ms, sys: 113 ms, total: 297 ms
Wall time: 3.42 s


In [36]:
stats

Start                       2015-01-01 00:00:00
End                         2019-06-21 00:00:00
Duration                     1632 days 00:00:00
Exposure [%]                            89.2157
Equity Final [$]                        7641.44
Equity Peak [$]                         8717.53
Return [%]                              9.16348
Buy & Hold Return [%]                   15.9635
Max. Drawdown [%]                       -15.691
Avg. Drawdown [%]                      -1.99011
Max. Drawdown Duration        370 days 00:00:00
Avg. Drawdown Duration         44 days 00:00:00
# Trades                                     19
Win Rate [%]                            42.1053
Best Trade [%]                          13.6186
Worst Trade [%]                        -4.28422
Avg. Trade [%]                         0.465672
Max. Trade Duration           376 days 00:00:00
Avg. Trade Duration            77 days 00:00:00
Expectancy [%]                          2.80236
SQN                                    0

In [29]:
# stats._strategy

<Strategy SmaCross(n1=25,n2=95)>

In [30]:
# バックテストの結果を可視化
bt.plot(plot_width= 600)

In [37]:
heatmap

n1  n2
5   10    7017.542080
    15    7630.200594
    20    6339.604794
    25    5529.724752
    30    5066.862959
             ...     
25  75    6653.867764
    80    6537.902440
    85    6697.473885
    90    7179.901587
    95    7641.443399
Length: 80, dtype: float64